In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils
from copy import deepcopy
%matplotlib inline  

Using Theano backend.


In [6]:
import holidays as holidays
import datetime as datetime

us_holidays = holidays.UnitedStates()  # or holidays.US()

date(2015, 1, 1) in us_holidays  # True
date(2015, 1, 2) in us_holidays  # False


'2014-01-01' in us_holidays  # True
'1/1/2014' in us_holidays  # True
1388597445 in us_holidays  # True (Unix timestamp)

us_holidays.get('2014-01-01')  # "New Year's Day"

custom_holidays = holidays.HolidayBase()
custom_holidays.append({"2015-01-01": "New Year's Day"})
custom_holidays.append(['2015-07-01', '07/04/2015'])
custom_holidays.append(date(2015, 12, 25))
date(2015, 1, 1) in custom_holidays  # True
date(2015, 1, 2) in custom_holidays  # False
'12/25/2015' in custom_holidays  # True

TypeError: 'module' object is not callable

In [7]:
df_raw=pd.read_csv('/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/train.csv')

In [8]:
df_raw['Dates']=pd.to_datetime(df_raw['Dates'])
df_raw.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [9]:
def isholiday(d) :
    return(d in us_holidays)
int(isholiday('2001-01-01 23:53:00'))

1

In [10]:
sum(df_raw['Dates'].apply(isholiday)==1)

28034

In [11]:
df_a=df_raw.loc[:,'Address']

In [12]:
print(sum(df_a.apply(lambda x: int(' AV' in x))))
print(sum(df_a.apply(lambda x: int(' BL' in x))))
print(sum(df_a.apply(lambda x: int(' CR' in x))))
print(sum(df_a.apply(lambda x: int(' CT' in x))))
print(sum(df_a.apply(lambda x: int(' DR' in x))))
print(sum(df_a.apply(lambda x: int(' BL' in x))))
print(sum(df_a.apply(lambda x: int(' LN' in x))))
print(sum(df_a.apply(lambda x: int(' PL' in x))))
print(sum(df_a.apply(lambda x: int(' PZ' in x))))
print(sum(df_a.apply(lambda x: int(' RD' in x))))
# print(sum(df_a.apply(lambda x: int(' ST' in x))))
print(sum(df_a.apply(lambda x: int(' TR' in x))))
print(sum(df_a.apply(lambda x: int(' HWY' in x))))
print(sum(df_a.apply(lambda x: int(' HY' in x))))
print(sum(df_a.apply(lambda x: int(' WY' in x))))
print(sum(df_a.apply(lambda x: int(' WAY' in x))))


200999
28751
1811
3285
15611
28751
1936
3392
2350
4120
3258
169
12251
7485
445


In [13]:
sum(df_a.apply(lambda x: int('/' in x)))

260818

In [14]:
df_raw['AV']=df_a.apply(lambda x: int(' AV' in x))
df_raw['BL']=df_a.apply(lambda x: int(' BL' in x))
df_raw['CR']=df_a.apply(lambda x: int(' CR' in x))
df_raw['CT']=df_a.apply(lambda x: int(' CT' in x))
df_raw['DR']=df_a.apply(lambda x: int(' DR' in x))
df_raw['BL']=df_a.apply(lambda x: int(' BL' in x))
df_raw['LN']=df_a.apply(lambda x: int(' LN' in x))
df_raw['PL']=df_a.apply(lambda x: int(' PL' in x))
df_raw['PZ']=df_a.apply(lambda x: int(' PZ' in x))
df_raw['RD']=df_a.apply(lambda x: int(' RD' in x))
df_raw['TR']=df_a.apply(lambda x: int(' TR' in x))
df_raw['HWY']=df_a.apply(lambda x: int((' HY' in x) or ('HWY' in x)) )
df_raw['WAY']=df_a.apply(lambda x: int((' WY' in x) or ('WAY' in x)) )

df_raw['IsIntersect'] = df_a.apply(lambda x: int('/' in x))



In [15]:
df_raw.columns

Index([u'Dates', u'Category', u'Descript', u'DayOfWeek', u'PdDistrict',
       u'Resolution', u'Address', u'X', u'Y', u'AV', u'BL', u'CR', u'CT',
       u'DR', u'LN', u'PL', u'PZ', u'RD', u'TR', u'HWY', u'WAY',
       u'IsIntersect'],
      dtype='object')

In [16]:
df_raw['Address']=df_raw['Address'].astype('category')
sum(df_raw['Address'].value_counts() > 1)
bool = df_raw['Address'].value_counts() >30
sum(df_raw['Address'].value_counts()[bool==True])

712968

In [25]:
addresses=sorted(df_raw["Address"].unique())
categories=sorted(df_raw["Category"].unique())
C_counts=df_raw.groupby(["Category"]).size()
A_C_counts=df_raw.groupby(["Address","Category"]).size()
A_counts=df_raw.groupby(["Address"]).size()
logodds={}
logoddsPA={}
MIN_CAT_COUNTS=2
default_logodds=np.log(C_counts/len(df_raw))-np.log(1.0-C_counts/float(len(df_raw)))



In [37]:
for addr in addresses:
    print(addr)
    # PA=A_counts[addr] /float(len(df_raw))
    PA=A_counts.loc[addr] /float(len(df_raw))
    logoddsPA[addr]=np.log(PA)-np.log(1.-PA)
    logodds[addr]=deepcopy(default_logodds)
    for cat in A_C_counts[addr].keys():
        if (A_C_counts.loc[addr,cat]>MIN_CAT_COUNTS) and A_C_counts.loc([addr,cat])<A_counts.loc[addr]:
            PA=A_C_counts.loc([addr,cat])/float(A_counts.loc[addr])
            logodds[addr][categories.index(cat)]=np.log(PA)-np.log(1.0-PA)
    logodds[addr]=pd.Series(logodds[addr])
    logodds[addr].index=range(len(categories))

0 Block of  HARRISON ST
0 Block of 10TH AV
0 Block of 10TH ST


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
default_logodds=np.log(C_counts/len(df_raw))-np.log(1.0-C_counts/float(len(df_raw)))

In [30]:
print(type(A_counts))

addr=A_counts.index[2]
print(addr)
print(A_counts.head())
#why is the following not working ?
A_counts.loc[addr]

<class 'pandas.core.series.Series'>
0 Block of 10TH ST
Address
0 Block of  HARRISON ST     1
0 Block of 10TH AV          5
0 Block of 10TH ST         49
0 Block of 11TH ST         48
0 Block of 12TH AV          7
dtype: int64


Address
0 Block of 10TH ST    49
dtype: int64

In [35]:
print(type(A_C_counts))

addr=A_C_counts.index[0]
cat='LARCENY/THEFT'
print(addr)
print(A_C_counts.head())
#why is the following not working ?
A_C_counts.loc[addr,cat]

<class 'pandas.core.series.Series'>
('0 Block of  HARRISON ST', 'LARCENY/THEFT')
Address                  Category      
0 Block of  HARRISON ST  LARCENY/THEFT     1
0 Block of 10TH AV       ASSAULT           2
                         FRAUD             1
                         OTHER OFFENSES    1
                         ROBBERY           1
dtype: int64


Address                  Category     
0 Block of  HARRISON ST  LARCENY/THEFT    1
dtype: int64